# ESTIMATION OF POVERTY RATES IN NEW YORK CITY

## DATA PREPROCESSING

In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



/kaggle/input/mldataset/incomedata.csv
/kaggle/input/incomedata/2015Income.csv


# Missing value imputation and data reduction(stage 1)

### 1. Read the csv file into a pandas dataframe and summarize the data 

In [2]:
df= pd.read_csv('../input/mldataset/incomedata.csv')

In [3]:
print("total rows in the dataset: ",df.shape[0] )
print("total features in the dataset: ",df.shape[1] )

total rows in the dataset:  4918
total features in the dataset:  475


### 2. Remove unwanted features with attribute subset selection 

In [4]:
cols = [c for c in df.columns if c.startswith('Estimate!!')]
lastcol=df.columns[-1]
firstcol=df.columns[0]
cols.append(lastcol)
cols.insert(0,firstcol)
cols.append('Percent Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate')

In [5]:
print("LIST OF ALL COLUMNS AFTER SUBSET SELECTION:")
df1=df[cols]
print(df1.columns)
print("#####################################################################################################")
print("\nTotal Columns Left:",df1.shape[1])


LIST OF ALL COLUMNS AFTER SUBSET SELECTION:
Index(['id', 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over',
       'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force',
       'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force',
       'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed',
       'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed',
       'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces',
       'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force',
       'Estimate!!EMPLOYMENT STATUS!!Civilian labor force',
       'Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate',
       ...
       'Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized popul

In [6]:
#Set tract Id as index of dataset
df1 = df1.set_index('id')
#Convert string numbers to number data type for all features
for i in df1.columns:
    df1[i]=pd.to_numeric(df1[i], errors='coerce')
df1.head()

,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over,...,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!With health insurance coverage!!With private health insurance,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!With health insurance coverage!!With public coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!No health insurance coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage!!With private health insurance,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage!!With public coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!No health insurance coverage,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families,Percent Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
id,,,,,,,,,,,,,,,,,,,,,
1400000US36001000100,1505,930,924,811,113,6,575,924,NaN,816,...,13,62,0,338,300,75,255,38,24.2,12.2
1400000US36001000200,3748,2519,2519,2277,242,0,1229,2519,NaN,2051,...,104,81,54,900,658,88,587,242,34.3,9.6
1400000US36001000300,4816,2894,2894,2598,296,0,1922,2894,NaN,2536,...,32,156,46,998,803,333,526,195,21.7,10.2
1400000US36001000401,2305,1000,1000,950,50,0,1305,1000,NaN,1434,...,28,22,22,172,146,91,73,26,2.0,5.0
1400000US36001000403,3756,2747,2747,2669,78,0,1009,2747,NaN,1875,...,43,21,0,483,483,287,262,0,2.4,2.8


### 3. Remove features with all missing values and duplicate features

In [7]:
# Find features with all missing values
nan_values = df1.isna()
nan_columns = nan_values.all()

columns_with_nan = df1.columns[nan_columns].tolist()
print("COLUMNS WHICH HAVE ALL NAN:", columns_with_nan)
df1=df1.drop('Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate',axis=1)
nan_values = df1.isna()
nan_columns = nan_values.all()

columns_with_nan = df1.columns[nan_columns].tolist()
print(columns_with_nan)

COLUMNS WHICH HAVE ALL NAN: ['Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate']
[]


In [9]:
#Function for finding duplicate features
def getDuplicateColumns(df): 
  
    # Create an empty set 
    duplicateColumnNames = set() 
      
    # Iterate through all the columns  
    # of dataframe 
    for x in range(df.shape[1]): 
          
        # Take column at xth index. 
        col = df.iloc[:, x] 
          
        # Iterate through all the columns in 
        # DataFrame from (x + 1)th index to 
        # last index 
        for y in range(x + 1, df.shape[1]): 
              
            # Take column at yth index. 
            otherCol = df.iloc[:, y] 
              
            # Check if two columns at x & y 
            # index are equal or not, 
            # if equal then adding  
            # to the set 
            if col.equals(otherCol): 
                duplicateColumnNames.add(df.columns.values[y])
                duplicateColumnNames.add(df.columns.values[x])
                  
    # Return list of unique column names  
    # whose contents are duplicates. 
    return list(duplicateColumnNames)
duplicate_cols = getDuplicateColumns(df1)
print(duplicate_cols)


['Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force', 'Estimate!!EMPLOYMENT STATUS!!Civilian labor force', 'Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over', 'Estimate!!OCCUPATION!!Civilian employed population 16 years and over', 'Estimate!!INDUSTRY!!Civilian employed population 16 years and over', 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed']


In [10]:
df[duplicate_cols]

,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over,Estimate!!OCCUPATION!!Civilian employed population 16 years and over,Estimate!!INDUSTRY!!Civilian employed population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed
0,924,924,811,811,811,811
1,2519,2519,2277,2277,2277,2277
2,2894,2894,2598,2598,2598,2598
3,1000,1000,950,950,950,950
4,2747,2747,2669,2669,2669,2669
...,...,...,...,...,...,...
4913,3005,3005,2835,2835,2835,2835
4914,2768,2768,2610,2610,2610,2610
4915,2405,2405,2325,2325,2325,2325
4916,1679,1679,1623,1623,1623,1623


In [11]:
#Drop duplicate columns
df1=df1.drop([duplicate_cols[0],duplicate_cols[2]],axis=1)

In [13]:
#print missing values for each feature
for col in list(df1.columns):
    print(col,df1[col].isna().sum())

Estimate!!EMPLOYMENT STATUS!!Population 16 years and over 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force 0
Estimate!!EMPLOYMENT STATUS!!Civilian labor force 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force!!Employed 0
Estimate!!EMPLOYMENT STATUS!!Own children of the householder under 6 years 0
Estimate!!EMPLOYM

### 4. Find rows (census tracts) which have a population of zero and remove them

In [14]:
tract0=df1[df1['Estimate!!EMPLOYMENT STATUS!!Population 16 years and over']==0]
df2=df1[df1['Estimate!!EMPLOYMENT STATUS!!Population 16 years and over']>0]
'''for col in list(df2.columns):
    print(col,df2[col].isna().sum())'''
temp=df2[df2.isnull().sum(axis=1)>10]


In [15]:
print("Total rows with 0 population: ",temp.shape[0])

Total rows with 0 population:  50


### 5. Fill missing values using Iterative Imputator

In [16]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
fill_NaN = IterativeImputer(max_iter=10, random_state=0)
imputed_DF = pd.DataFrame(fill_NaN.fit_transform(df2))
imputed_DF.columns = df2.columns
imputed_DF.index = df2.index

/opt/conda/lib/python3.7/site-packages/sklearn/impute/_iterative.py:670: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [17]:
imputed_DF.head()

,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force,...,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!With health insurance coverage!!With private health insurance,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!With health insurance coverage!!With public coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!No health insurance coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage!!With private health insurance,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage!!With public coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!No health insurance coverage,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families,Percent Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
id,,,,,,,,,,,,,,,,,,,,,
1400000US36001000100,1505.0,930.0,811.0,113.0,6.0,575.0,924.0,816.0,521.0,521.0,...,13.0,62.0,0.0,338.0,300.0,75.0,255.0,38.0,24.2,12.2
1400000US36001000200,3748.0,2519.0,2277.0,242.0,0.0,1229.0,2519.0,2051.0,1549.0,1549.0,...,104.0,81.0,54.0,900.0,658.0,88.0,587.0,242.0,34.3,9.6
1400000US36001000300,4816.0,2894.0,2598.0,296.0,0.0,1922.0,2894.0,2536.0,1405.0,1405.0,...,32.0,156.0,46.0,998.0,803.0,333.0,526.0,195.0,21.7,10.2
1400000US36001000401,2305.0,1000.0,950.0,50.0,0.0,1305.0,1000.0,1434.0,574.0,574.0,...,28.0,22.0,22.0,172.0,146.0,91.0,73.0,26.0,2.0,5.0
1400000US36001000403,3756.0,2747.0,2669.0,78.0,0.0,1009.0,2747.0,1875.0,1268.0,1268.0,...,43.0,21.0,0.0,483.0,483.0,287.0,262.0,0.0,2.4,2.8


In [18]:
df2.head()

,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force,Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force,...,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!With health insurance coverage!!With private health insurance,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!With health insurance coverage!!With public coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!In labor force!!Unemployed!!No health insurance coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage!!With private health insurance,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!With health insurance coverage!!With public coverage,Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!Civilian noninstitutionalized population 19 to 64 years!!Not in labor force!!No health insurance coverage,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families,Percent Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
id,,,,,,,,,,,,,,,,,,,,,
1400000US36001000100,1505,930,811,113,6,575,924,816,521,521,...,13,62,0,338,300,75,255,38,24.2,12.2
1400000US36001000200,3748,2519,2277,242,0,1229,2519,2051,1549,1549,...,104,81,54,900,658,88,587,242,34.3,9.6
1400000US36001000300,4816,2894,2598,296,0,1922,2894,2536,1405,1405,...,32,156,46,998,803,333,526,195,21.7,10.2
1400000US36001000401,2305,1000,950,50,0,1305,1000,1434,574,574,...,28,22,22,172,146,91,73,26,2.0,5.0
1400000US36001000403,3756,2747,2669,78,0,1009,2747,1875,1268,1268,...,43,21,0,483,483,287,262,0,2.4,2.8


In [19]:
#print missing values remaining in each feature
for col in list(imputed_DF.columns):
    print(col,imputed_DF[col].isna().sum())

Estimate!!EMPLOYMENT STATUS!!Population 16 years and over 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces 0
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force 0
Estimate!!EMPLOYMENT STATUS!!Civilian labor force 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force 0
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force!!Employed 0
Estimate!!EMPLOYMENT STATUS!!Own children of the householder under 6 years 0
Estimate!!EMPLOYM

In [20]:
print("Total missing values: ",imputed_DF.isna().sum().sum())

Total missing values:  0


In [21]:
print("Shape of cleaned dataset")
print("rows:",imputed_DF.shape[0]," , Columns:",imputed_DF.shape[1])

Shape of cleaned dataset
rows: 4851  , Columns: 117


### 6. Export cleaned data to CSV file

In [22]:
imputed_DF.to_csv('./imputedData2018.csv')